<a href="https://colab.research.google.com/github/mokshdoshi2005/Python-Library-Compatibility-Manager/blob/main/lib_manager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# SIMPLE GOOGLE COLAB TEST
# Copy and paste this into a Google Colab cell

# Step 1: Install dependencies
!pip install packaging requests -q

# Step 2: Import and define the simplified LibraryManager class
import subprocess
import sys
import requests
from typing import List, Dict, Optional, Tuple
from packaging import specifiers, version
from importlib import metadata
import tempfile

class LibraryManager:
    def __init__(self):
        self.pypi_base_url = "https://pypi.org/pypi"
        self.installed_packages = self._get_installed_packages()

    def _get_installed_packages(self) -> Dict[str, str]:
        """Get currently installed packages and their versions"""
        installed = {}
        try:
            distributions = metadata.distributions()
            for dist in distributions:
                name = dist.metadata.get('Name', '').lower()
                version_str = dist.metadata.get('Version', '')
                if name and version_str:
                    installed[name] = version_str
        except Exception as e:
            print(f"Warning: Error getting installed packages: {e}")
        return installed

    def _get_package_info(self, package_name: str) -> Optional[Dict]:
        """Fetch package information from PyPI"""
        try:
            response = requests.get(f"{self.pypi_base_url}/{package_name}/json", timeout=10)
            if response.status_code == 200:
                return response.json()
            return None
        except requests.RequestException as e:
            print(f"Error fetching info for {package_name}: {e}")
            return None

    def _get_dependencies(self, package_name: str) -> List[str]:
        """Extract dependencies from package info"""
        info = self._get_package_info(package_name)
        if not info:
            return []
        return info["info"].get("requires_dist", []) or []

    def _parse_requirement(self, req_string: str) -> Tuple[str, str]:
        """Parse requirement string to get package name and version spec"""
        if not req_string:
            return "", ""

        req_string = req_string.split(';')[0].strip()
        req_string = req_string.split('[')[0].strip()

        for op in ['>=', '<=', '==', '!=', '>', '<', '~=']:
            if op in req_string:
                parts = req_string.split(op, 1)
                return parts[0].strip(), f"{op}{parts[1].strip()}"

        return req_string.strip(), ""

    def check_package_compatibility(self, packages: List[str]) -> Dict[str, List[str]]:
        """Check compatibility between packages"""
        conflicts = {}
        all_deps = {}

        for package in packages:
            deps = self._get_dependencies(package)
            for dep in deps:
                dep_name, dep_version = self._parse_requirement(dep)
                if dep_name:
                    if dep_name not in all_deps:
                        all_deps[dep_name] = []
                    all_deps[dep_name].append((package, dep_version))

        # Find conflicting version requirements
        for dep_name, requirements in all_deps.items():
            if len(requirements) > 1:
                version_specs = [req[1] for req in requirements if req[1]]
                if len(set(version_specs)) > 1:
                    conflicts[dep_name] = [f"{req[0]} requires {req[1]}" for req in requirements]

        return conflicts

    def list_sample_packages(self, limit: int = 15):
        """List a sample of installed packages"""
        print(f"Sample of installed packages ({limit} out of {len(self.installed_packages)}):")
        for i, (name, ver) in enumerate(sorted(self.installed_packages.items())):
            if i >= limit:
                break
            print(f"  {name}: {ver}")

    def get_package_info_summary(self, package_name: str):
        """Get and display package information summary"""
        info = self._get_package_info(package_name)
        if info:
            print(f"📦 {package_name}:")
            print(f"   Version: {info['info']['version']}")
            print(f"   Summary: {info['info']['summary']}")
            print(f"   Author: {info['info']['author']}")
            deps = info['info'].get('requires_dist', [])
            print(f"   Dependencies: {len(deps) if deps else 0}")
            return info
        else:
            print(f"❌ Could not fetch info for {package_name}")
            return None

    def detailed_conflict_analysis(self, packages: List[str]):
            """Perform detailed conflict analysis with explanations"""
            print("=" * 80)
            print(f"DETAILED CONFLICT ANALYSIS")
            print("=" * 80)

            conflicts = self.check_package_compatibility(packages)

            print(f"\n📊 SUMMARY:")
            print("-" * 40)
            if conflicts:
                print(f"❌ Found {len(conflicts)} potential conflicts:")
                for i, (dep, conflict_list) in enumerate(conflicts.items(), 1):
                    print(f"\n{i}. Conflict in '{dep}':")
                    for conflict in conflict_list:
                        print(f"   • {conflict}")

                    # Try to suggest resolution
                    print(f"   💡 Resolution: Choose compatible versions or use virtual environments")
            else:
                print("✅ No conflicts detected - packages should work together!")

            return conflicts
# Step 3: Create manager and run tests
print("🚀 Creating Library Manager...")
manager = LibraryManager()

print(f"\n✅ Successfully initialized! Found {len(manager.installed_packages)} packages")

# Test 1: Show sample packages
print("\n" + "="*60)
print("TEST 1: SAMPLE INSTALLED PACKAGES")
print("="*60)
manager.list_sample_packages()

# Test 2: Get info for a specific package
print("\n" + "="*60)
print("TEST 2: PACKAGE INFO")
print("="*60)
manager.get_package_info_summary('numpy')

# Test 3: Check compatibility
print("\n" + "="*60)
print("TEST 3: COMPATIBILITY CHECK")
print("="*60)
test_packages = ['requests', 'urllib3']
print(f"Checking compatibility for: {test_packages}")
conflicts = manager.check_package_compatibility(test_packages)
if conflicts:
    print("⚠️  Potential conflicts detected:")
    for dep, conflict_list in conflicts.items():
        print(f"  {dep}:")
        for conflict in conflict_list:
            print(f"    - {conflict}")
else:
    print("✅ No compatibility issues detected")

# Test 4: Check if specific packages are installed
print("\n" + "="*60)
print("TEST 4: CHECK INSTALLED STATUS")
print("="*60)
check_packages = ['numpy', 'pandas', 'matplotlib', 'torch', 'tensorflow']
for pkg in check_packages:
    if pkg in manager.installed_packages:
        print(f"✅ {pkg}: {manager.installed_packages[pkg]}")
    else:
        print(f"❌ {pkg}: Not installed")

print("\n🎉 All tests completed!")
print("\n💡 Try these commands in new cells:")
print("manager.get_package_info_summary('pandas')")
print("manager.check_package_compatibility(['numpy', 'scipy', 'matplotlib'])")
print("len(manager.installed_packages)")

🚀 Creating Library Manager...

✅ Successfully initialized! Found 637 packages

TEST 1: SAMPLE INSTALLED PACKAGES
Sample of installed packages (15 out of 637):
  absl-py: 1.4.0
  accelerate: 1.9.0
  aiofiles: 24.1.0
  aiohappyeyeballs: 2.6.1
  aiohttp: 3.12.14
  aiosignal: 1.4.0
  alabaster: 1.0.0
  albucore: 0.0.24
  albumentations: 2.0.8
  ale-py: 0.11.2
  altair: 5.5.0
  annotated-types: 0.7.0
  antlr4-python3-runtime: 4.9.3
  anyio: 4.9.0
  anywidget: 0.9.18

TEST 2: PACKAGE INFO
📦 numpy:
   Version: 2.3.2
   Summary: Fundamental package for array computing in Python
   Author: Travis E. Oliphant et al.
   Dependencies: 0

TEST 3: COMPATIBILITY CHECK
Checking compatibility for: ['requests', 'urllib3']
✅ No compatibility issues detected

TEST 4: CHECK INSTALLED STATUS
✅ numpy: 2.0.2
✅ pandas: 2.2.2
✅ matplotlib: 3.10.0
✅ torch: 2.6.0+cu124
✅ tensorflow: 2.18.0

🎉 All tests completed!

💡 Try these commands in new cells:
manager.get_package_info_summary('pandas')
manager.check_package_

In [2]:
# Test 1: Create manager and check installed packages
manager = LibraryManager()
print(f"Total packages: {len(manager.installed_packages)}")

# Test 2: Check specific package info
manager.get_package_info_summary('requests')

# Test 3: Check compatibility between packages
conflicts = manager.check_package_compatibility(['numpy', 'pandas', 'matplotlib'])
print("Conflicts found:" if conflicts else "No conflicts!")

# Test 4: Check what packages are installed
important_packages = ['numpy', 'pandas', 'matplotlib', 'seaborn', 'sklearn']
for pkg in important_packages:
    status = "✅ Installed" if pkg in manager.installed_packages else "❌ Not installed"
    version = f" ({manager.installed_packages.get(pkg, 'N/A')})" if pkg in manager.installed_packages else ""
    print(f"{pkg}: {status}{version}")

Total packages: 637
📦 requests:
   Version: 2.32.4
   Summary: Python HTTP for Humans.
   Author: Kenneth Reitz
   Dependencies: 6
Conflicts found:
numpy: ✅ Installed (2.0.2)
pandas: ✅ Installed (2.2.2)
matplotlib: ✅ Installed (3.10.0)
seaborn: ✅ Installed (0.13.2)
sklearn: ❌ Not installed


In [8]:
# Test specific packages
manager.detailed_conflict_analysis(['tensorflow', 'torch'])

# Test web development stack
manager.detailed_conflict_analysis(['django', 'flask', 'fastapi'])

# Test data science packages
manager.detailed_conflict_analysis(['pandas==1.3.0', 'numpy==1.24.0'])

DETAILED CONFLICT ANALYSIS

📊 SUMMARY:
----------------------------------------
❌ Found 13 potential conflicts:

1. Conflict in 'opt-einsum':
   • tensorflow requires >=2.3.2
   • torch requires >=3.3
   💡 Resolution: Choose compatible versions or use virtual environments

2. Conflict in 'typing-extensions':
   • tensorflow requires >=3.6.6
   • torch requires >=4.10.0
   💡 Resolution: Choose compatible versions or use virtual environments

3. Conflict in 'nvidia-cublas-cu12':
   • tensorflow requires ==12.5.3.2
   • torch requires ==12.6.4.1
   💡 Resolution: Choose compatible versions or use virtual environments

4. Conflict in 'nvidia-cuda-cupti-cu12':
   • tensorflow requires ==12.5.82
   • torch requires ==12.6.80
   💡 Resolution: Choose compatible versions or use virtual environments

5. Conflict in 'nvidia-cuda-nvrtc-cu12':
   • tensorflow requires ==12.5.82
   • torch requires ==12.6.77
   💡 Resolution: Choose compatible versions or use virtual environments

6. Conflict in 'nvid

{}